<a href="https://colab.research.google.com/github/hope04302/freeSearch/blob/main/main_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 한국어 혐오 발언 탐지

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 필요 라이브러리 설치

In [ ]:
!pip install torch==2.0.1 torchvision~=0.15.1 https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-2.0-cp310-cp310-linux_x86_64.whl
!pip install torchaudio~=2.0.1

!pip install transformers
!pip install datasets
!pip install torchmetrics
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.9/162.9 MB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 97.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 65.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 88.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 38.6 MB/s eta 0:00:00
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.1.0+cu118
    Uninstalling torchaudio-2.1.0+cu118:
      Successfully uninstalled torchaudio-2.1.0+cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 6.6 MB/s eta 0:00:00
  

## 데이터셋 로드

- 학습, 검증, 테스트 데이터셋 준비
- 라벨 정보

      class_label:
        names:
          0: origin
          1: physical
          2: politics
          3: profanity
          4: age
          5: gender
          6: race
          7: religion
          8: not_hate_speech

In [ ]:
from datasets import load_dataset
from sklearn.preprocessing import MultiLabelBinarizer
from torchmetrics import Accuracy, F1Score, HammingDistance, AUROC, ExactMatch

from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback

import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp

import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
NUM_LABELS = 9
BATCH_SIZE = 32
EPOCHS = 4
CORES = 1       # tpu vm = 1

# =========================================
# bert-base-multilingual-cased, monologg/koelectra-base-v3-discriminator, monologg/kobert, snunlp/KR-BERT-char16424
name_card = 'snunlp/KR-BERT-char16424'
model_name = "KR-BERT-c/type_2"
model_path = f'/content/drive/MyDrive/models/{model_name}'
# =========================================

In [ ]:
device = xm.xla_device()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(name_card, do_lower_case=False)

In [ ]:
def tokenize_function(examples):

    model_input = tokenizer(examples['text'], max_length=128, truncation=True, padding="max_length")

    mlb = MultiLabelBinarizer(classes=[0,1,2,3,4,5,6,7,8])
    one_hot_labels = mlb.fit_transform(examples['label'])

    model_input['label'] = one_hot_labels
    model_input['n_label'] = [sum(one_hot_label) for one_hot_label in one_hot_labels]

    return model_input

In [ ]:
dataset = load_dataset("jeanlee/kmhas_korean_hate_speech")

Generating train split:   0%|          | 0/78977 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8776 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/21939 [00:00<?, ? examples/s]

In [ ]:
train, valid, test = dataset.map(tokenize_function, batched=True).values()

Map:   0%|          | 0/78977 [00:00<?, ? examples/s]

Map:   0%|          | 0/8776 [00:00<?, ? examples/s]

Map:   0%|          | 0/21939 [00:00<?, ? examples/s]

## 모델 로드

In [ ]:
# ===============================================

class Model(nn.Module):

    def __init__(self):

        super(Model, self).__init__()
        self.model = AutoModel.from_pretrained(name_card)

        self.classifier = nn.Linear(self.model.config.hidden_size, NUM_LABELS)
        self.n_classifier = nn.Linear(self.model.config.hidden_size, 1)

        self.intent_loss = nn.BCEWithLogitsLoss()
        self.n_loss = nn.MSELoss()


    def forward(self, input_ids, attention_mask, token_type_ids=None, labels=None, n_label=None):

        model_input= {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
        }

        if token_type_ids is not None:
            model_input['token_type_ids'] = token_type_ids

        hidden = self.model(**model_input)

        if hasattr(hidden, 'pooler_output'):
            cls = hidden.pooler_output
        else:
            cls = hidden.last_hidden_state[:, 0, :]

        hidden = hidden.last_hidden_state[:, 1:, :]

        intent_logit = self.classifier(hidden.mean(dim=1))
        n_logit = self.n_classifier(cls)[:, 0]

        intent_loss = self.intent_loss(intent_logit, labels)
        num_loss = self.n_loss(n_logit, n_label.float())

        loss = intent_loss + num_loss

        model_output = {
            'loss': loss,
            'intent_logit': intent_logit,
            'n_logit': n_logit,
        }

        return model_output

# ===============================================

## 성능 지표 계산

In [ ]:
def compute_metrics(eval_pred):

    # ==================================================

    intent_logits, n_logits = eval_pred.predictions
    intent_labels, n_labels = eval_pred.label_ids

    intent_logits = torch.Tensor(intent_logits)
    labels = torch.Tensor(intent_labels).long()

    n_logits = torch.Tensor(n_logits)
    n_labels = torch.Tensor(n_labels).long()

    n_preds = torch.round(n_logits).long()

    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(intent_logits))

    preds = torch.zeros(size = probs.size())

    # preds[probs >= 0.5] = 1

    for i in range(len(probs)):
        for _ in range(n_preds[i]):
            _max = torch.argmax(probs[i])
            preds[i][_max] = 1
            probs[i][_max] = 0

    # ==================================================

    accuracy = Accuracy(task='multilabel', num_labels=NUM_LABELS)
    f1_macro = F1Score(task="multilabel", num_labels=NUM_LABELS, average='macro')
    f1_micro = F1Score(task="multilabel", num_labels=NUM_LABELS, average='micro')
    f1_weight = F1Score(task="multilabel", num_labels=NUM_LABELS, average='weighted')
    em = ExactMatch(task='multiclass', num_classes=2)
    auroc = AUROC(task='multilabel', num_labels=NUM_LABELS, average='micro')
    hamming = HammingDistance(task="multiclass", num_classes=2)
    n_int_accuracy = Accuracy(task='multiclass', num_classes=NUM_LABELS)

    each_f1_score = F1Score(task='multilabel', num_labels=NUM_LABELS, average=None)

    print(each_f1_score(preds, labels))

    metrics = {'accuracy': accuracy(preds, labels),
               'f1_macro': f1_macro(preds, labels),
               'f1_micro': f1_micro(preds, labels),
               'f1_weighted': f1_weight(preds, labels),
               'auroc': auroc(preds, labels),
               'hamming_loss': hamming(preds, labels),
               'em': em(preds, labels),
               'n_int_accuracy': n_int_accuracy(n_preds, n_labels)}

    return metrics

## 모델 학습

In [ ]:
training_args = TrainingArguments(

    output_dir=model_path,
    do_train=True,
    do_eval=True,
    evaluation_strategy="epoch",
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    logging_strategy="epoch",
    logging_dir='./logs',
    learning_rate=2e-5,
    run_name="v1",
    seed=42,
    tpu_num_cores=CORES,

    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model='eval_f1_micro',
    save_total_limit=1,
    label_names=['labels', 'n_label'],

)

In [ ]:
def _mp_train_fn(index=None, frags=None):

    model = Model().to(device)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train,
        eval_dataset=valid,
        compute_metrics=compute_metrics,
        # callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
    )

    trainer.train()
    trainer.save_model()

In [ ]:
xmp.spawn(_mp_train_fn, args=(), nprocs=CORES, start_method='fork')

Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,F1 Micro,F1 Weighted,Auroc,Hamming Loss,Em,N Int Accuracy,Runtime,Samples Per Second,Steps Per Second
1,0.229900,0.191423,0.957764,0.701754,0.834145,0.833525,0.909587,0.042236,0.781677,0.910210,74.594300,117.972000,3.687000
2,0.143100,0.153056,0.965145,0.747417,0.861177,0.859449,0.920279,0.034855,0.823952,0.936304,41.784300,210.606000,6.581000
3,0.109900,0.157425,0.964676,0.754675,0.860136,0.860243,0.921737,0.035324,0.819508,0.932885,38.301200,229.758000,7.180000


tensor([0.7147, 0.7633, 0.8016, 0.8727, 0.7769, 0.6942, 0.0000, 0.8046, 0.8879])
tensor([0.7775, 0.7802, 0.8305, 0.9008, 0.8092, 0.7474, 0.1429, 0.8351, 0.9032])
tensor([0.7768, 0.7907, 0.8314, 0.8997, 0.8154, 0.7644, 0.1935, 0.8194, 0.9007])


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,F1 Micro,F1 Weighted,Auroc,Hamming Loss,Em,N Int Accuracy,Runtime,Samples Per Second,Steps Per Second
1,0.229900,0.191423,0.957764,0.701754,0.834145,0.833525,0.909587,0.042236,0.781677,0.910210,74.594300,117.972000,3.687000
2,0.143100,0.153056,0.965145,0.747417,0.861177,0.859449,0.920279,0.034855,0.823952,0.936304,41.784300,210.606000,6.581000
3,0.109900,0.157425,0.964676,0.754675,0.860136,0.860243,0.921737,0.035324,0.819508,0.932885,38.301200,229.758000,7.180000
4,0.087400,0.156994,0.965575,0.759932,0.862739,0.861930,0.920784,0.034425,0.824977,0.937215,38.352700,229.449000,7.170000


tensor([0.7773, 0.7790, 0.8374, 0.9008, 0.8227, 0.7595, 0.2222, 0.8369, 0.9036])


## 모델 예측

In [ ]:
def _mp_test_fn(index=None, frags=None):

    state_dict = torch.load(f"/{model_path}/pytorch_model.bin")
    model = Model().to(device)
    model.load_state_dict(state_dict=state_dict)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train,
        eval_dataset=test,
        compute_metrics=compute_metrics,
    )

    print(trainer.evaluate())

In [ ]:
xmp.spawn(_mp_test_fn, args=(), nprocs=CORES, start_method='fork')

tensor([0.7963, 0.7866, 0.8335, 0.9083, 0.8229, 0.7345, 0.2558, 0.8436, 0.9020])
{'eval_loss': 0.15786175429821014, 'eval_accuracy': 0.9650495648384094, 'eval_f1_macro': 0.764834463596344, 'eval_f1_micro': 0.8617006540298462, 'eval_f1_weighted': 0.8609548807144165, 'eval_auroc': 0.9196491241455078, 'eval_hamming_loss': 0.034950435161590576, 'eval_em': 0.8203199505805969, 'eval_n_int_accuracy': 0.932129979133606, 'eval_runtime': 158.1085, 'eval_samples_per_second': 138.841, 'eval_steps_per_second': 4.339}


In [ ]:
def _mp_test_fn(index=None, frags=None):

    state_dict = torch.load(f"/{model_path}/pytorch_model.bin")
    model = Model().to(device)
    model.load_state_dict(state_dict=state_dict)

    test_dict = {1: [], 2: [], 3: [], 4: []}
    for i in test:
        n = i["n_label"]
        test_dict[n].append(i)


    for i in test_dict.values():

        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train,
            eval_dataset=i,
            compute_metrics=compute_metrics,
        )

        print(trainer.evaluate())

In [ ]:
xmp.spawn(_mp_test_fn, args=(), nprocs=CORES, start_method='fork')

tensor([0.7841, 0.7324, 0.8109, 0.8754, 0.7987, 0.6256, 0.3265, 0.8361, 0.9061])
{'eval_loss': 0.11362475156784058, 'eval_accuracy': 0.9685518145561218, 'eval_f1_macro': 0.7439671754837036, 'eval_f1_micro': 0.8605833649635315, 'eval_f1_weighted': 0.8621025085449219, 'eval_auroc': 0.9269872903823853, 'eval_hamming_loss': 0.031448185443878174, 'eval_em': 0.8491008877754211, 'eval_n_int_accuracy': 0.9705498814582825, 'eval_runtime': 77.7043, 'eval_samples_per_second': 247.09, 'eval_steps_per_second': 7.722}


tensor([0.8373, 0.8966, 0.8701, 0.9486, 0.8631, 0.8406, 0.2143, 0.9045, 0.0000])
{'eval_loss': 0.40309253334999084, 'eval_accuracy': 0.9464633464813232, 'eval_f1_macro': 0.708345890045166, 'eval_f1_micro': 0.8725257515907288, 'eval_f1_weighted': 0.8804494738578796, 'eval_auroc': 0.9029041528701782, 'eval_hamming_loss': 0.05353665351867676, 'eval_em': 0.6588285565376282, 'eval_n_int_accuracy': 0.6985891461372375, 'eval_runtime': 9.2258, 'eval_samples_per_second': 267.076, 'eval_steps_per_second': 8.346}


tensor([0.7170, 0.7952, 0.8669, 0.9404, 0.8333, 0.7523, 0.0000, 0.7500, 0.0000])
{'eval_loss': 0.8952777981758118, 'eval_accuracy': 0.8999999761581421, 'eval_f1_macro': 0.6283388733863831, 'eval_f1_micro': 0.8317214846611023, 'eval_f1_weighted': 0.825372040271759, 'eval_auroc': 0.8603448271751404, 'eval_hamming_loss': 0.10000002384185791, 'eval_em': 0.341379314661026, 'eval_n_int_accuracy': 0.4172413647174835, 'eval_runtime': 1.4178, 'eval_samples_per_second': 225.708, 'eval_steps_per_second': 7.053}


tensor([0.6000, 0.8000, 0.8571, 0.9778, 0.7273, 0.8824, 0.0000, 0.0000, 0.0000])
{'eval_loss': 1.6846060752868652, 'eval_accuracy': 0.8533333539962769, 'eval_f1_macro': 0.5382829308509827, 'eval_f1_micro': 0.8114285469055176, 'eval_f1_weighted': 0.7891150712966919, 'eval_auroc': 0.8389999866485596, 'eval_hamming_loss': 0.14666664600372314, 'eval_em': 0.07999999821186066, 'eval_n_int_accuracy': 0.11999999731779099, 'eval_runtime': 0.2941, 'eval_samples_per_second': 108.791, 'eval_steps_per_second': 3.4}
